<h3> The Problem </h3>

My background before I entered the Masters in Business Analytics program was Seattle University was in event production and I continue to produce events to this day. Ticketing Providers take a large percentage of every transaction and I sought out alternate options to avoid what are often exporbitant fees.
While I was impressed by Linktree's 'Request Payment' Feature from a Patron/UX perspective, there were some disadvantages. As someone who throws events often with hundreds of attendees, this information wasn't provided
in a centralized database anywhere either in an online portal or as an exportable csv (at least at the time of this project). While using Linktree, which take a miniscule fee of 0.5% for Pro Plan Transactions would save at a minimumum hundreds of dollars in ticketing fees, this was a major issue and I produced a solution to workaround this.

<h3> The Solution </h3>
Every time a user submitted a payment, I received an email notification. While the information was buried inside the body of the email, it was there.
Google's Takeout feature provides an MBOX File containing all email information. From here, I converted this MBOX File to a CSV, but all the important information was contained in just a single cell on each row
and this information was structured in a way that didn't allow for easier parsing of information through a program such as Excel.

Having this information structured in a more coherent manner would allow for easier tracking of financial information for the event and allowed for a simpler list that could be used at the door when checking
patrons in. In addition, providing a streamlined CSV containing information such as the day of the week of a ticket purchase and the time would provide a suitable foundation for conducting further analysis. 

In [4]:
import pandas as pd

data = pd.read_csv(r'C:\Users\nickb\Documents\Research_TheNight\Great Day Night For It\Sep42020_Great Day For It\TicketExport1.csv')

In [5]:
data.head()

,Date,Subject,Message
0,27/08/2022 12:18,Request received! via Linktree,Linktree ( https://url1741.linktr.ee/ls/click?...
1,27/08/2022 12:15,Request received! via Linktree,Linktree ( https://url1741.linktr.ee/ls/click?...
2,26/08/2022 20:57,Request received! via Linktree,Linktree ( https://url1741.linktr.ee/ls/click?...
3,26/08/2022 18:38,Request received! via Linktree,Linktree ( https://url1741.linktr.ee/ls/click?...
4,26/08/2022 18:36,Request received! via Linktree,Linktree ( https://url1741.linktr.ee/ls/click?...


In [5]:
# Extracting Only Linktree Payment Emails from Email Export
data = data[data['Subject'] == 'Request received! via Linktree']

# Dropping Subject Colunmn
data = data.drop(columns=['Subject'])

# Extracting Tier Value
data['Ticket Tier'] = data['Message'].str.split('Tier', 1).str[1].str[1]

# Creating Email Column
data['Email'] = data['Message'].str.split('From', 1).str[1:]

import re as re

def find_email(text):
    email = re.findall(r'[\w\.-]+@[\w\.-]+',str(text))
    return ",".join(email)

data['Email'] = data['Email'].apply(lambda x: find_email(x))

# Removes first n character
data['Email'] = data['Email'].str[1:]

# Remove Linktree Emails...
# NEED TO FIGURE OUT HOW 
data['Email'] = data['Email'].map(lambda x: x.rstrip(',support@linktr.ee'))

# Creating Name Column
data['Ticket Name'] = data['Message'].str.split('Special instructions or details', 1).str[0:2].str[1]

# Creating Amount Paid Column
data['Amount Paid'] = data['Message'].str.split('$', 1).str[0:2].str[1]
data['Amount Paid'] = data['Amount Paid'].str.split('\(USD').str[0]

data['Date'] = pd.to_datetime(data['Date'])

data['Date Purchased'] = data['Date'].dt.date
data['Time Purchased'] = data['Date'].dt.time
data['DOW Purchased'] = data['Date'].dt.day_name()

data.sort_values(by=['Date Purchased', 'Time Purchased'])

# TO DO.... make this for others as well
# Lynn Nakamura had a missing value, this assigns it there
data.loc[data['Date'] == '2022-08-22 22:43:00','Amount Paid'] = 25.67

data["Amount Paid"] = pd.to_numeric(data['Amount Paid'], errors='ignore')

# Gets Rid of New Lines, Back Slashes, etc.
data = data.replace('\r','', regex=True)
data = data.replace('\n','', regex=True)
data = data.replace('-','', regex=True)

# Removes View at linktree information cluttering column
data['Ticket Name'] = data['Ticket Name'].str.split('View').str[0]

data['Email'] = data['Email'].str.split(',').str[0]

In [14]:
data.loc[data['Message'].str.contains("print", case=False)]

,Date,Message,Ticket Tier,Email,Ticket Name,Amount Paid,Date Purchased,Time Purchased,DOW Purchased
42,2022-08-24 08:50:00,Linktree ( https://url1741.linktr.ee/ls/click?...,3,chasemartin63@gmail.com,None,32.85,2022-08-24,08:50:00,Wednesday
55,2022-08-17 21:32:00,Linktree ( https://url1741.linktr.ee/ls/click?...,3,rachelvick2@gmail.com,n/a,32.85,2022-08-17,21:32:00,Wednesday
57,2022-08-17 10:37:00,Linktree ( https://url1741.linktr.ee/ls/click?...,3,sarahmuir31@gmail.com,N/A,32.85,2022-08-17,10:37:00,Wednesday
96,2022-04-08 22:35:00,Linktree ( https://url1741.linktr.ee/ls/click?...,1,anniefholden@gmail.com,:),32.85,2022-04-08,22:35:00,Friday
101,2022-04-08 13:35:00,Linktree ( https://url1741.linktr.ee/ls/click?...,1,usmanajamil@gmail.com,Usmanusmanajamil@gmail.com,32.85,2022-04-08,13:35:00,Friday
103,2022-04-08 11:54:00,Linktree ( https://url1741.linktr.ee/ls/click?...,1,boydst@plu.edu,Samuel Boyd boydst@plu.edu,32.85,2022-04-08,11:54:00,Friday
114,2022-03-08 23:45:00,Linktree ( https://url1741.linktr.ee/ls/click?...,1,Maxpknyc@gmail.com,❤️❤️❤️,32.85,2022-03-08,23:45:00,Tuesday
134,2022-03-08 18:17:00,Linktree ( https://url1741.linktr.ee/ls/click?...,2,muazzezm@gmail.com,Muazzez Miramuazzezm@gmail.com,27.72,2022-03-08,18:17:00,Tuesday
141,2022-03-08 16:51:00,Linktree ( https://url1741.linktr.ee/ls/click?...,2,diamondasimpson@gmail.com,Diamond Simpson diamondasimpson@gmail.com,27.72,2022-03-08,16:51:00,Tuesday
145,2022-03-08 15:51:00,Linktree ( https://url1741.linktr.ee/ls/click?...,2,ewolf41@gmail.com,none,27.72,2022-03-08,15:51:00,Tuesday


In [12]:
# illustrations of the event artwork were sold as an add-on for tickets
data.loc[data['Message'].str.contains("illustration", case=False)]

,Date,Message,Ticket Tier,Email,Ticket Name,Amount Paid,Date Purchased,Time Purchased,DOW Purchased
55,2022-08-17 21:32:00,Linktree ( https://url1741.linktr.ee/ls/click?...,3,rachelvick2@gmail.com,n/a,32.85,2022-08-17,21:32:00,Wednesday
57,2022-08-17 10:37:00,Linktree ( https://url1741.linktr.ee/ls/click?...,3,sarahmuir31@gmail.com,N/A,32.85,2022-08-17,10:37:00,Wednesday
96,2022-04-08 22:35:00,Linktree ( https://url1741.linktr.ee/ls/click?...,1,anniefholden@gmail.com,:),32.85,2022-04-08,22:35:00,Friday
134,2022-03-08 18:17:00,Linktree ( https://url1741.linktr.ee/ls/click?...,2,muazzezm@gmail.com,Muazzez Miramuazzezm@gmail.com,27.72,2022-03-08,18:17:00,Tuesday
141,2022-03-08 16:51:00,Linktree ( https://url1741.linktr.ee/ls/click?...,2,diamondasimpson@gmail.com,Diamond Simpson diamondasimpson@gmail.com,27.72,2022-03-08,16:51:00,Tuesday
145,2022-03-08 15:51:00,Linktree ( https://url1741.linktr.ee/ls/click?...,2,ewolf41@gmail.com,none,27.72,2022-03-08,15:51:00,Tuesday
169,2022-03-08 14:13:00,Linktree ( https://url1741.linktr.ee/ls/click?...,2,kiyomers@gmail.com,Kiyomi NunezKiyomers@gmail.com,27.72,2022-03-08,14:13:00,Tuesday
173,2022-03-08 13:14:00,Linktree ( https://url1741.linktr.ee/ls/click?...,2,au5tinnn@aol.com,Austin LevensonAustin.lev@gmail.com,27.72,2022-03-08,13:14:00,Tuesday
174,2022-03-08 13:09:00,Linktree ( https://url1741.linktr.ee/ls/click?...,2,katharinelandahl@gmail.com,None,27.72,2022-03-08,13:09:00,Tuesday
193,2022-03-08 11:57:00,Linktree ( https://url1741.linktr.ee/ls/click?...,2,dahummus@gmail.com,Preferred Name: Dylan DavisEmail: Dahummus@gm...,27.72,2022-03-08,11:57:00,Tuesday


In [136]:
# Exporting CSV
data.to_csv (r'C:\Users\nickb\Documents\Research_TheNight\Sep42020_Great Day For It\PowderTicketingAug16\powder_ticketing_afterprocessing.csv', index = False, header=True)